<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Compute_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
#!git clone https://github.com/informatics-isi-edu/eye-ai-tools.git
#!pip install ./eye-ai-tools --upgrade
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools@main' --upgrade --force


  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-43cre3b8
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-43cre3b8
  Resolved https://github.com/fair-research/bdbag to commit 8ee285059caa71a0789179b8d57b03010ac00e7f
  Preparing metadata (setup.py) ... done
  Created wheel for bdbag: filename=bdbag-1.7.1-py2.py3-none-any.whl size=72695 sha256=792d41b704e679cb6ee82fc7bccad6cc71dd4861ee403b486673ba97b2e8aa38
  Stored in directory: /tmp/pip-ephem-wheel-cache-pxnqgxbi/wheels/e6/37/a1/ed335483cf88440ed32b8f918cc20fa31b2ee2c81dd9504f7a
Successfully built bdbag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [5]:
# login to DERIVA via Globus Auth
DEBUG_MODE = True #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'
!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


You are already logged in.


Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [6]:
# @title Configure Grader groups
grader_list = ["Benjamin Xu", "Kendra Hong", "Kent Nguyen"] # @param
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}


In [7]:
# @title Configure Process and
process_tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Metadata = "Test Diagnosis Generation" # @param {type:"string"}
Github_URL="" # @param {type:"string"}
Process_Tag = "" # @param {type: "string"}
Description = "" # @param {type: "string"}
Github_Checksum = "" # @param {type: "string"}



Diagnosis_Tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Description = "Generate new diagnosis based on a group of graders: "+str(grader_list)
synonyms = [] # @param


In [8]:
from ml_utils import EyeAI
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_number = 'eye-ai')

In [10]:
# @title Define generation rule
def CDR_func(values):
    return values.mean()

def Diag_func(values):
    # Use the mode function to get the most frequent diagnosis value
    mode_diagnosis = values.mode()

    # If there are multiple modes, take the first one
    if not mode_diagnosis.empty:
        return mode_diagnosis.iloc[0]
    else:
        # Handle the case where there is no mode (e.g., all values are unique)
        return 'Unknown'

def ImageQuality_func(values):
    mode_quality = values.mode()
    if not mode_quality.empty:
        return mode_quality.iloc[0]
    else:
        return "Unknown"

In [11]:
# @title Extract the diagnosis in the catalog and generate new Diagnosis
all_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
result = EA.compute_diagnosis(all_diag[all_diag["Full_Name"].isin(grader_list)],
                                 Diag_func,
                                 CDR_func,
                                 ImageQuality_func)

In [12]:
# @title Insert New Process_Tag, Proecess, and Diagnosis_Tag
new_process_tag_RID = EA.add_term('Process_Tag', name=process_tag_name, description=grader_list, synonyms=[], exist_ok=False)
Process_RID = EA.add_process(Metadata, Github_URL, new_process_tag_RID, Description, Github_Checksum)

new_diag_tag_RID = EA.add_term('Diagnosis_Tag', Diagnosis_Tag_name, Description, exist_ok=False)


# New Section

In [13]:
entities = EA.insert_new_diagnosis(result, new_diag_tag_RID, Process_RID)